Web Form Autofiller for University Major Recommendation (Type 1)
---

**Overall logic**
1. We use a web crawler to automatically fill in an online questionnaire based on the student's responses to a series of personality and academic related questions.
2. During the autofill process, we record the crawler's selected fields and the generated major recommendation output. We repeat this process many times to collect our own dataset.
3. Using our collected dataset, we train our own major recommendation model.

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 1. Autofiller of Online Questionnaire
**Online questionnaire source(s)**

["What's My Major Quiz" by Loyola University Chicago](https://www.luc.edu/undergrad/academiclife/whatsmymajorquiz/#)
- 40 yes/no short questions - $2^{40}$ possible combinations
- Recommends multiple majors

## Setup

In [13]:
# import selenium related packages
from time import sleep
from selenium import webdriver

In [14]:
# define webdriver paths
CHROME_DRIVER_PATH = './drivers/chromedriver'

In [15]:
# define urls to be crawled
LUC_QUIZ_URL = 'https://www.luc.edu/undergrad/academiclife/whatsmymajorquiz/#'

## Helper functions

In [30]:
# helper function
# step 1: click yes/no 40 times on the online questionnaire to reach the final recommendation
import random
def luc_random_click(browser):
    options = ['yes', 'no']
    chosen_option = random.choice(options) # generate random option between 'yes' and 'no'

    btn = browser.find_element_by_id(chosen_option) # locate the radio button
    btn.click() # click the radio button on the webpage

In [ ]:
# helper function
# step 2: 

In [31]:
# define chrome browser
browser = webdriver.Chrome(CHROME_DRIVER_PATH)

# launch browser
browser.get(LUC_QUIZ_URL)
sleep(2)

for _ in range(40):
    luc_random_click(browser)
    sleep(1)

In [9]:
browser.get(LUC_QUIZ_URL)

sleep(5)

browser.close()